In [1]:
import torch
import torchvision
import cv2
import numpy as np
import pandas as pd
import os
import time
import random
import copy
from Data_PreProcess import Data_preprocess as dp
import json
from data_loader import DataGenerate,collate
from torch.utils.data import Dataset, DataLoader
from Generate_Anchor import Generate_Anchor
from torchvision import models
from Net3 import Net
from Eval import eval as eval_model

In [2]:
#for thres in [0.8,0.9,0.95,0.5,0.6,0.7,0.3]:
#r,p = eval_model(model,threshold=0.5)

In [3]:
def get_split_line(anchor_nums=3, batch_size=2):
    num1 = 128*128*batch_size*anchor_nums
    num2 = 64*64*batch_size*anchor_nums + num1
    num3 = 32*32*batch_size*anchor_nums + num2
    return [[0,num1],[num1,num2],[num2,num3]]

In [4]:
batch_size = 2
input_size = 864
anchor_info = {
    "anchor_size_3" : [[350, 350],[425, 425],[475, 475],[400, 300],[300, 400],[450, 350],[350, 450]],
    "anchor_size_1" : [[ 20,  20],[ 30,  30],[ 45,  45],[ 70,  70],[100, 100],[100,  60],[ 60, 100]],
    "anchor_size_2" : [[150, 150],[225, 225],[300, 300],[225, 150],[150, 225],[100, 200],[200, 100]],
    "feat_size_1":(input_size//8,input_size//8),
    "feat_size_2":(input_size//16,input_size//16),
    "feat_size_3":(input_size//32,input_size//32)
}
#FPN 各层生成的Anchor独立，方便修改
anchor_generator_1 = Generate_Anchor(batch_size=batch_size,stride=8, feat_size=anchor_info["feat_size_1"],anchor_size=anchor_info["anchor_size_1"])
anchor_generator_2 = Generate_Anchor(batch_size=batch_size,stride=16, feat_size=anchor_info["feat_size_2"],anchor_size=anchor_info["anchor_size_2"])
anchor_generator_3 = Generate_Anchor(batch_size=batch_size,stride=32, feat_size=anchor_info["feat_size_3"],anchor_size=anchor_info["anchor_size_3"])
train_generate = DataLoader(DataGenerate(batch_size=batch_size, input_size=input_size), batch_size = batch_size, drop_last = True,collate_fn=collate)

model = Net(input_size=864,batch_size=batch_size, anchor_info=anchor_info)
model.load_state_dict(torch.load(r"E:\cancer\model\model_include3\eps=3.t7"),strict = False)
model = model.cuda()
#eval_model(model, input_size = input_size, thres=0.4)

E:\anaconda\Anaconda\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [5]:
def get_anchor(data):
    label_1,label_valid_1, loc_1,cls_1 = anchor_generator_1.getAnchor(data)
    label_2,label_valid_2, loc_2,cls_2 = anchor_generator_2.getAnchor(data)
    label_3,label_valid_3, loc_3,cls_3 = anchor_generator_3.getAnchor(data)
    return torch.from_numpy(np.hstack([label_1,label_2,label_3])).cuda().float(),\
            torch.from_numpy(np.hstack([label_valid_1,label_valid_2,label_valid_3])).cuda(),\
            torch.from_numpy(np.vstack([loc_1,loc_2,loc_3])).cuda().float()

In [6]:
#torch.save(model.state_dict(), r"E:\cancer\model\test\eps=11.t7")

In [7]:
from Generate_Anchor import generate_prior_anchor,get_prior_anchor
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
eval_prior_anchor = get_prior_anchor(input_size=input_size)
model = model.train()
pos_num = []
smooth = 0.05
loss_hist_loc_s1 = []
loss_hist_cls_s1 = []
loss_hist_loc_s2 = []
loss_hist_cls_s2 = []
loss_hist_ph = []
loss_hist_pp = []
loss_hist_hh = []
learning_rate=0.0001
lr = np.ones(4)
for eps in range(11,20):
    start = time.time()
    learning_rate = learning_rate*0.5
    print("learning_rate = %f"%learning_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for idx,data in enumerate(train_generate):
        #cv2.waitKey(100)
        loss_cls_s1,loss_loc_s1,loss_cls_s2,loss_loc_s2, pos_hard, pos_pos, hard_hard = model(data[0],data[1],threshold=0.5)
        loss = loss_cls_s1*lr[1]*1.3 + loss_loc_s1*lr[0] + loss_cls_s2 * lr[3]*1.3 + loss_loc_s2 * lr[2] + (pos_hard+pos_pos+hard_hard)*0.3
        loss_hist_loc_s1.append(loss_loc_s1.item())
        loss_hist_cls_s1.append(loss_cls_s1.item())
        loss_hist_loc_s2.append(loss_loc_s2.item())
        loss_hist_cls_s2.append(loss_cls_s2.item())
        loss_hist_ph.append(pos_hard.item())
        loss_hist_pp.append(pos_pos.item())
        loss_hist_hh.append(hard_hard.item())
        if((idx%500 == 0) and idx>0):
            loss_mean = np.array([np.mean(loss_hist_loc_s1),np.mean(loss_hist_cls_s1),np.mean(loss_hist_loc_s2),np.mean(loss_hist_cls_s2)])
            print("loss_loc = %.5f | loss_cls = %.5f | loss_loc_s2 = %.5f | loss_cls_s2 = %.5f "%(
                loss_mean[0],loss_mean[1],loss_mean[2],loss_mean[3]))
            print("pos_hard = %.5f | pos_pos = %.5f | hard_hard = %.5f"%(np.mean(loss_hist_ph), np.mean(loss_hist_pp), np.mean(loss_hist_hh)))
            loss_hist_loc_s1 = []
            loss_hist_cls_s1 = []
            loss_hist_loc_s2 = []
            loss_hist_cls_s2 = []
            loss_hist_ph = []
            loss_hist_pp = []
            loss_hist_hh = []

            if (idx>1000):  # 根据各类loss的下降比例 调整学习率系数
                dif = (last_loss - loss_mean)/last_loss
                dif = np.exp(dif)
                dif = np.sum(dif)/(dif + smooth)
                lr = dif/np.mean(dif)
                lr = np.clip(lr,0.1,10)

            last_loss = loss_mean
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("time ",time.time() - start)
    torch.save(model.state_dict(), r"E:\cancer\model\test\eps=%d.t7"%eps)
    eval_model(model, input_size = input_size, s2_threshold=0.55)
    model.train(True)

learning_rate = 0.000050
trainning set =  5945
loss_loc = 0.08032 | loss_cls = 0.03924 | loss_loc_s2 = 0.05327 | loss_cls_s2 = 0.01476 
pos_hard = 0.78936 | pos_pos = 0.52976 | hard_hard = 0.31101
loss_loc = 0.07985 | loss_cls = 0.03505 | loss_loc_s2 = 0.05679 | loss_cls_s2 = 0.01314 
pos_hard = 0.70845 | pos_pos = 0.51380 | hard_hard = 0.27218
loss_loc = 0.07925 | loss_cls = 0.04256 | loss_loc_s2 = 0.05379 | loss_cls_s2 = 0.01677 
pos_hard = 0.79037 | pos_pos = 0.53164 | hard_hard = 0.29449
loss_loc = 0.07669 | loss_cls = 0.03411 | loss_loc_s2 = 0.05305 | loss_cls_s2 = 0.01795 
pos_hard = 0.79472 | pos_pos = 0.54359 | hard_hard = 0.31543
loss_loc = 0.07724 | loss_cls = 0.03254 | loss_loc_s2 = 0.05611 | loss_cls_s2 = 0.01898 
pos_hard = 0.71887 | pos_pos = 0.54801 | hard_hard = 0.27609
time  2805.6521322727203


TypeError: eval() got an unexpected keyword argument 's2_threshold'